# Lab: Spark Machine Learning

In this lab you will be digging into the world of credit markets and peer-to-peer lending. Loan decisions from financial institutions depend on a vast trove of data on borrower characteristics, market trends, and more. The best way to harness the vast data is with machine learning. An [article by the Brookings Institution](https://www.brookings.edu/research/credit-denial-in-the-age-of-ai/) discusses the impact of AI on loan decisions and higlights that “If there are data out there on you, there is probably a way to integrate it into a credit model.” 

We will use Spark Machine Learning to build a machine learning pipeline that can model and predict credit worthiness. The data we will be using comes from Lending Club, a company that conducted peer-to-peer lending from 2007 to 2020. There are over 2.9 million loans in the dataset. Each row of data about the loan, credit history, and the success of the loan. Credit history variables include  number of accounts, past missed payments, occupation, years of experience, etc. You can read about the source data [from Kaggle](https://www.kaggle.com/ethon0426/lending-club-20072020q1).

## Understanding your data

Find all your Spark related environment variables, and pyspark using the `findspark.init()` function:

In [1]:
import findspark
findspark.init()

Create your SparkSession. You are only going to create a `SparkSession`, not a `SparkContext`.

In [2]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col, lit
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("lendingclub").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/19 00:04:07 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Make sure your SparkSession is active:

In [5]:
spark

## 1. Read in the data

The data is in the format of an Apache Parquet file. This file format is optimized for use on clusters, which you can read more about [here](https://drill.apache.org/docs/parquet-format/). You will be working with this file using [Spark DataFrame API and Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html) as we did in the SparkSQL lab.

Start off by moving the data from the central bucket to your personal bucket.

In [8]:
!aws s3 cp s3://bigdatateaching/lendingclub s3://anly502-fall-2022-yl1353 --recursive

copy: s3://bigdatateaching/lendingclub/lendingclub_loan_data.parquet to s3://anly502-fall-2022-yl1353/lendingclub_loan_data.parquet


Load in the Parquet file using [Generic load functions for SparkSQL](https://spark.apache.org/docs/latest/sql-programming-guide.html#data-sources), which is located at `s3://[[YOUR-BUCKET-NAME]]/lendingclub/lendingclub_loan_data.parquet`.

Create a DataFrame called `df_in`, which should contain many rows and over 140 columns.

In [9]:
df_in = spark.read.parquet('s3://anly502-fall-2022-yl1353/lendingclub_loan_data.parquet')

Whenever you load in a new dataset, you will always do the following three actions:

    1. Get a count of the rows
    2. Print the schema of the data
    3. View the first few rows of the data (there are a lot of columns so just look at 3)

Do these steps now!

In [10]:
df_in.count()

22/10/19 00:06:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


2925492

In [11]:
df_in.printSchema()

root
 |-- id: string (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- funded_amnt: double (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- earliest_cr

In [12]:
df_in.take(3)

[Row(id='1077501', loan_amnt=5000.0, funded_amnt=5000.0, funded_amnt_inv=4975.0, term=' 36 months', int_rate=' 10.65%', installment=162.87, grade='B', sub_grade='B2', emp_title=None, emp_length='10+ years', home_ownership='RENT', annual_inc=24000.0, verification_status='Verified', issue_d='Dec-2011', loan_status='Fully Paid', pymnt_plan='n', url='https://lendingclub.com/browse/loanDetail.action?loan_id=1077501', purpose='credit_card', title='Computer', zip_code='860xx', addr_state='AZ', dti=27.65, delinq_2yrs=0.0, earliest_cr_line='Jan-1985', fico_range_low=735.0, fico_range_high=739.0, inq_last_6mths=1.0, mths_since_last_delinq=None, mths_since_last_record=None, open_acc=3.0, pub_rec=0.0, revol_bal=13648.0, revol_util='83.7%', total_acc=9.0, initial_list_status='f', out_prncp=0.0, out_prncp_inv=0.0, total_pymnt=5863.1551866952, total_pymnt_inv=5833.84, total_rec_prncp=5000.0, total_rec_int=863.16, total_rec_late_fee=0.0, recoveries=0.0, collection_recovery_fee=0.0, last_pymnt_d='Jan-2

## 2.1 Clean data to prepare for machine learning

No dataset is ever ready for modeling without cleaning. There are several steps required to get the data ready:
    
1. Select only the columns - `['int_rate', 'term', 'loan_status', 'home_ownership', 'tot_cur_bal', 'annual_inc', 'grade', 'bc_util']` and then drop if there are any coded NA values left `.dropna()` [read more here on the method](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.dropna.html).

These columns mean:
* `int_rate`: interest rate
* `term`: length of the loan
* `loan_status`: loan paid, in progress, or charged off
* `home_ownership`: home ownership status
* `tot_cur_bal`: total current balance on all credit accounts
* `grade`: The grade of the loan
* `bc_util`: bank card utilization rate

2. Extract the numbers from the column `int_rate`. You will have to first use `select` and `show` methods to view what the data looks like. Then you can use the [regex replace technique](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.regexp_replace.html) `regexp_replace` or a python-based user defined function method. **There are both XX.XX% and X.XX% rate types in this column**. Finally, cast `int_rate` to float.

4. Filter to only rows with `loan_status` of "Fully Paid" or "Charged Off". This means that the loan went to "maturity" so we can determine if the loan was successful (fully paid) or unsuccessful (charged off).

5. Check the potential values for `term`, `home_ownership`, and `grade`. What are the possible categories of each?

6. Drop rows with unusual values for on the `home_ownership` variable . There are several categories that are small and have ambiguous meaning. There are three to drop and three to keep.

7. Report the number of rows left in your DataFrame.

1. Selecting columns

In [52]:
df = df_in.select(['int_rate', 'term', 'loan_status', 'home_ownership', 'tot_cur_bal', 'annual_inc', 'grade', 'bc_util'])

In [53]:
df = df.dropna()

2. String extraction and data type conversion

In [54]:
# always look at the data before
df.select('int_rate').show(10)

+--------+
|int_rate|
+--------+
|  10.99%|
|  10.99%|
|  10.99%|
|   7.62%|
|  12.85%|
|   6.62%|
|   8.90%|
|  16.24%|
|   7.62%|
|  11.99%|
+--------+
only showing top 10 rows



In [55]:
from pyspark.sql.functions import regexp_replace
df = df.withColumn('int_rate', regexp_replace('int_rate', '%', '').cast('float'))

In [56]:
# always look at the data after
df.select('int_rate').show(10)

+--------+
|int_rate|
+--------+
|   10.99|
|   10.99|
|   10.99|
|    7.62|
|   12.85|
|    6.62|
|     8.9|
|   16.24|
|    7.62|
|   11.99|
+--------+
only showing top 10 rows



3. Drop loans that have not completed, keeping only `Charged Off` or `Fully Paid`

In [57]:
# a count function by distinct value is quite important in pyspark!
df.groupby('loan_status').count().collect()

[Row(loan_status='In Grace Period', count=9863),
 Row(loan_status='Late (31-120 days)', count=15932),
 Row(loan_status='Issued', count=2047),
 Row(loan_status='Fully Paid', count=1423664),
 Row(loan_status='Default', count=422),
 Row(loan_status='Charged Off', count=347950),
 Row(loan_status='Current', count=1018833),
 Row(loan_status='Late (16-30 days)', count=2678)]

In [63]:
df = df.filter(df.loan_status.isin(['Charged Off','Fully Paid']))

In [65]:
df.show(10)

+--------+----------+-----------+--------------+-----------+----------+-----+-------+
|int_rate|      term|loan_status|home_ownership|tot_cur_bal|annual_inc|grade|bc_util|
+--------+----------+-----------+--------------+-----------+----------+-----+-------+
|   10.99| 36 months| Fully Paid|          RENT|     7137.0|   60000.0|    B|   15.9|
|   10.99| 36 months| Fully Paid|      MORTGAGE|     4136.0|   39600.0|    B|   16.1|
|   10.99| 36 months| Fully Paid|           OWN|   114834.0|   55000.0|    B|   53.9|
|    7.62| 36 months| Fully Paid|      MORTGAGE|   200314.0|   96500.0|    A|   83.5|
|   12.85| 36 months| Fully Paid|          RENT|    17672.0|   88000.0|    B|   87.7|
|    6.62| 36 months| Fully Paid|      MORTGAGE|   267646.0|  105000.0|    A|   25.0|
|     8.9| 36 months| Fully Paid|      MORTGAGE|   272492.0|   63000.0|    A|   79.1|
|   16.24| 36 months| Fully Paid|          RENT|     5759.0|   28000.0|    C|   96.0|
|    7.62| 36 months| Fully Paid|      MORTGAGE|   799

4. Check possible values for categorical columns

In [67]:
df.printSchema()

root
 |-- int_rate: float (nullable = true)
 |-- term: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- tot_cur_bal: double (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- bc_util: double (nullable = true)



In [68]:
df.groupby('term').count().collect()

[Row(term=' 60 months', count=446669), Row(term=' 36 months', count=1324945)]

In [69]:
df.groupby('home_ownership').count().collect()

[Row(home_ownership='MORTGAGE', count=877147),
 Row(home_ownership='ANY', count=1158),
 Row(home_ownership='NONE', count=45),
 Row(home_ownership='RENT', count=695491),
 Row(home_ownership='OWN', count=197729),
 Row(home_ownership='OTHER', count=44)]

In [70]:
df.groupby('grade').count().collect()

[Row(grade='D', count=261779),
 Row(grade='B', count=519897),
 Row(grade='G', count=10004),
 Row(grade='F', count=34720),
 Row(grade='C', count=507755),
 Row(grade='E', count=110052),
 Row(grade='A', count=327407)]

5. Drop rows with explicitly missing data

In [72]:
df = df.dropna()

6. Report how much data remains after your cleaning steps

In [73]:
df.count()

1771614

**Run this cell to export 5 rows of your data to a json for grading**

In [74]:
import json
json.dump({'df' : df.limit(5).toPandas().to_dict('records'),
           'home_counts' : [x.asDict() for x in df.groupby('home_ownership').count().collect()]
          },
          fp = open('data-soln-1.json','w'))

## 2.2. Split data into train, test, and split

In this section you will split your data into: train, test and predict datasets. Create three splits of `df_in` (train, test, predict) by using the `randomSplit` method. Read more about the method [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.randomSplit.html).

In [75]:
train_data, test_data, predict_data = df.randomSplit([0.8, 0.18, 0.02], 24)

After splitting into three datasets, report the number of rows for each split.

In [76]:
print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 1417006


Number of testing records : 319270


Number of prediction records : 35338


### 2.2: Create pipeline and train a model

For this lab, your job is to build a model that classifies the `loan_status`. In this section you will create a machine learning pipeline and then train the model. The next cell imports all the packages you will need:

In [77]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric indices. Look at the DataFrame schema to determine which ones you need to convert. Use the `StringIndexer` estimator. You need to create a transformer for each column you want to modify because the "model" has to find out how many distinct options there are for the variable in order to assign consistent integer values.

The format of the command will be `StringIndexer(inputCol="[INPUT COL NAME]", outputCol="[OUTPUT COL NAME]")`

In [94]:
#print the schema again
train_data.printSchema()

root
 |-- int_rate: float (nullable = true)
 |-- term: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- tot_cur_bal: double (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- bc_util: double (nullable = true)



In [88]:
stringIndexer_loan = StringIndexer(inputCol="loan_status", outputCol="loan_success")
stringIndexer_term = StringIndexer(inputCol="term", outputCol="term_ix")
stringIndexer_home = StringIndexer(inputCol="home_ownership", outputCol="home_ownership_ix")
stringIndexer_grade = StringIndexer(inputCol="grade", outputCol="grade_ix")

Try looking at the values for one of the re-encoded columns using the `labels` method. Does it work?

In [80]:
stringIndexer_grade.labels

AttributeError: 'StringIndexer' object has no attribute 'labels'

To see the values of the labels, the estimator needs to be fitted and made a transformer first. You can do so by using the `fit` method. Try that now:

In [97]:
from pyspark.ml.feature import StringIndexer
indexed = stringIndexer_home.fit(train_data).transform(train_data)

In [98]:
indexed_t = stringIndexer_grade.fit(indexed).transform(indexed)

In [102]:
indexed_a = stringIndexer_loan.fit(indexed_t).transform(indexed_t)

In [103]:
indexed_b = stringIndexer_term.fit(indexed_a).transform(indexed_a)

In this step, we need to convert our index variables that have more than two levels. Use the function OneHotEncoder [(read more here)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.OneHotEncoder.html). The format is the same as the StringIndexer.

In [104]:
onehot_home = OneHotEncoder(inputCol="home_ownership_ix", outputCol="home_ownership_vec")
onehot_grade = OneHotEncoder(inputCol="grade_ix", outputCol="grade_vec")

In [105]:
f1 = onehot_home.fit(indexed_b).transform(indexed_b)
f2 = onehot_grade.fit(f1).transform(f1)

In the following step, create a feature vector by combining all string features together using the `vectorAssembler` method:

In [133]:
vectorAssembler_features = VectorAssembler(
    inputCols=["term_ix", "home_ownership_vec", "grade_vec"], 
    outputCol= "features")

What does the resulting transformer look like? There is no data here because the transformer is just the blueprint for the change in the DataFrame

In [134]:
feature_vectorized = vectorAssembler_features.transform(f2)
feature_vectorized.show(10)

+--------+----------+-----------+--------------+-----------+----------+-----+-------+-----------------+--------+------------+-------+------------------+-------------+--------------------+
|int_rate|      term|loan_status|home_ownership|tot_cur_bal|annual_inc|grade|bc_util|home_ownership_ix|grade_ix|loan_success|term_ix|home_ownership_vec|    grade_vec|            features|
+--------+----------+-----------+--------------+-----------+----------+-----+-------+-----------------+--------+------------+-------+------------------+-------------+--------------------+
|    5.31| 36 months|Charged Off|      MORTGAGE|    11518.0|   42000.0|    A|   23.2|              0.0|     2.0|         1.0|    0.0|     (5,[0],[1.0])|(6,[2],[1.0])|(12,[1,8],[1.0,1.0])|
|    5.31| 36 months|Charged Off|      MORTGAGE|    13153.0|   65400.0|    A|   70.3|              0.0|     2.0|         1.0|    0.0|     (5,[0],[1.0])|(6,[2],[1.0])|(12,[1,8],[1.0,1.0])|
|    5.31| 36 months|Charged Off|      MORTGAGE|    15169.0|

Next, define the estimator you want to use for classification. You will build a Random Forest using the `RandomForestClassifier` [estimator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html). This function is also where you would adjust hyperparameters for your model.

In [135]:
rf = RandomForestClassifier(labelCol="loan_success", featuresCol="features", numTrees=50)

Finally, indexed labels back to original labels.

In [136]:
labelConverter = IndexToString(inputCol="prediction", 
                               outputCol="predictedLoanStatus", 
                               labels= ['Fully Paid','Charged Off'])

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [137]:
pipeline_rf = Pipeline(stages=[stringIndexer_loan, 
                               stringIndexer_term, 
                               stringIndexer_home, 
                               stringIndexer_grade, 
                               onehot_home,
                               onehot_grade,
                               vectorAssembler_features, 
                               rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [131]:
train_data.printSchema()

root
 |-- int_rate: float (nullable = true)
 |-- term: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- tot_cur_bal: double (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- bc_util: double (nullable = true)



In [132]:
train_data.show(10)

+--------+----------+-----------+--------------+-----------+----------+-----+-------+
|int_rate|      term|loan_status|home_ownership|tot_cur_bal|annual_inc|grade|bc_util|
+--------+----------+-----------+--------------+-----------+----------+-----+-------+
|    5.31| 36 months|Charged Off|      MORTGAGE|    11518.0|   42000.0|    A|   23.2|
|    5.31| 36 months|Charged Off|      MORTGAGE|    13153.0|   65400.0|    A|   70.3|
|    5.31| 36 months|Charged Off|      MORTGAGE|    15169.0|   85000.0|    A|   22.2|
|    5.31| 36 months|Charged Off|      MORTGAGE|    16313.0|  130000.0|    A|   21.4|
|    5.31| 36 months|Charged Off|      MORTGAGE|    16848.0|   45000.0|    A|   13.3|
|    5.31| 36 months|Charged Off|      MORTGAGE|    24527.0|   78000.0|    A|   50.5|
|    5.31| 36 months|Charged Off|      MORTGAGE|    24991.0|   18000.0|    A|   19.4|
|    5.31| 36 months|Charged Off|      MORTGAGE|    29641.0|  161000.0|    A|   29.8|
|    5.31| 36 months|Charged Off|      MORTGAGE|    37

In [138]:
model_rf = pipeline_rf.fit(train_data)

What does `model_rf` look like? What is it, an estimator or a transformer?

In [139]:
print(model_rf)

PipelineModel_eb205cf38a7b


What happens when you send the train data through the transform method?

In [142]:
pipeline_rf.fit(train_data).transform(train_data)

DataFrame[int_rate: float, term: string, loan_status: string, home_ownership: string, tot_cur_bal: double, annual_inc: double, grade: string, bc_util: double, loan_success: double, term_ix: double, home_ownership_ix: double, grade_ix: double, home_ownership_vec: vector, grade_vec: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double, predictedLoanStatus: string]

## 3.1 Model Test Results

In this section, we will evaluate the **model accuracy**. This requires using the evaluator `MulticlassClassificationEvaluator`, which you can read about in the doc [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html). To evaluate the model, use **test data**.

First, let's transform the test data using our model pipeline

In [143]:
predictions = model_rf.transform(test_data)

Next, let's run the `MulticlassClassificationEvaluator` by passing in the label column (acutal result), prediction column (from our model), and the metric we want to calculate.

In [144]:
evaluatorRF = MulticlassClassificationEvaluator(labelCol="loan_success", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

In [145]:
evaluatorRF = MulticlassClassificationEvaluator(labelCol="loan_success", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

In [146]:
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.803931
Test Error = 0.196069


Was the test error similar to the train error? Run the same accuracy metrics on the training data

In [147]:
predictions_new = model_rf.transform(train_data)

In [148]:
evaluatorRF = MulticlassClassificationEvaluator(labelCol="loan_success", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions_new)

In [149]:
print("Accuracy = %g" % accuracy)
print("Train Error = %g" % (1.0 - accuracy))

Accuracy = 0.803498
Train Error = 0.196502


## 3.2 Confusion Matrix

In this section, we will build the confusion matrix from the model. We will use sci-kit learn to build the confusion matrix. We first have to extract the predicted label and the true label columns. These are the numeric binary form of the data. Finally, send them through the `confusion_matrix` method.

In [150]:
from sklearn.metrics import confusion_matrix

In [151]:
y_pred=predictions.select("prediction").collect()
y_orig=predictions.select("loan_success").collect()

In [152]:
cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[256671      0]
 [ 62599      0]]


## 3.3 ROC Curve

In this section, we will evaluate the **model area under the curve**. This requires using the evaluator `BinaryClassificationEvaluator`, which you can read about in the doc [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.BinaryClassificationEvaluator.html). To evaluate the model, use **test data**.

In [153]:
evaluatorRF = BinaryClassificationEvaluator(labelCol="loan_success", rawPredictionCol="prediction", metricName="areaUnderROC")
roc_result = evaluatorRF.evaluate(predictions)
roc_result

0.5

**Run this cell to export model metrics to a json for grading**

In [154]:
import json
json.dump({'roc' : roc_result,
           'cm' : str(cm),
           'acc' : accuracy
          },
          fp = open('data-soln-2.json','w'))

# MAKE SURE YOU STOP YOUR EMR CLUSTER!

In [155]:
spark.stop()